# Nova Tarefa - Implantação

Preencha aqui com detalhes sobre a tarefa.<br>
### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo-real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [ ]:
%%writefile Model.py
from wordcloud import WordCloud
import panadas as pd
import numpy as np

# adicione imports aqui...

class Model:
    def __init__(self):
        self.loaded = False

    def load(self):
        # utilize esta função para carregar e inicializar modelos
        artifacts = joblib.load("/tmp/data/wordcloud.joblib")
        self.wordcloud_parameters = artifacts["wordcloud_parameters"]

        self.wc = WordCloud
        self.loaded = True
        print("Loaded model")

    def predict(self, X, feature_names, meta=None):
        if not self.loaded:
            self.load()
        
        input_data = X
        images = []
        
        if isinstance(X, bytes):
            string = str(X, 'utf-8')
            data = StringIO(string)
            try:
                input_data = pd.read_csv(data)
            except:
                input_data = string
            
        if type(input_data) == str:
            wc.generate(input_data)
            images.append(wc.to_image())
        
        elif type(input_data) == pd.DataFrame:
            for text in input_data["text"]:
                wc.generate(text)
                images.append(wc.to_image())
             
        # a função predict recebe um parâmetro X (numpy array, str ou bytes)
        # e deve retornar o resultado da tarefa (numpy array, list, str ou bytes)
        return X